In [ ]:
import findspark
findspark.init('spark-3.5.1-bin-hadoop3')
from pyspark import *
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField, StringType, IntegerType, DateType, TimestampType, LongType
from pyspark.sql.types import ArrayType, DoubleType, BooleanType, DecimalType
from pyspark.sql.functions import regexp_extract, split, from_unixtime, col, avg, min, max, desc
from pyspark.sql.functions import grouping, explode, array_contains
from pyspark.sql.functions import mean, stddev, skewness, kurtosis
import matplotlib.pyplot as plt
import pandas as pd


spark = SparkSession.builder.appName("analytics").getOrCreate()

In [ ]:
# Cargar el CSV en un DataFrame de Spark
df = spark.read.csv("datos.csv", header=True, inferSchema=True)

# Mostrar el esquema del DataFrame
df.printSchema()

## Medidas de tendencia central

Spark Docs:

mean: [mean](https://spark.apache.org/docs/3.1.1/api/python/reference/api/pyspark.sql.functions.mean.html)

median (approxQuantile): [pyspark.sql.DataFrame.approxQuantile](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrame.approxQuantile.html)

mode: [mode](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.functions.mode.html)

In [ ]:
mean_calificacion = df.select(mean("Calificacion")).collect()[0][0]
print(mean_calificacion)

median_calificacion = df.approxQuantile("Calificacion", [0.5], 0.0)[0]
print(median_calificacion)

mode_calificacion = df.groupBy("Calificacion").count().orderBy("count", ascending=False).first()[0]
print(mode_calificacion)

## Medidas de dispersión

stddev: [stddev](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.functions.stddev.html)

In [ ]:
stddev_calificacion = df.select(stddev("Calificacion")).collect()[0][0]
stddev_calificacion

## Medidas de forma

skewness: [skewness](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.functions.skewness.html) 

kurtosis: [kurtosis](https://spark.apache.org/docs/3.1.3/api/python/reference/api/pyspark.sql.functions.kurtosis.html)

In [ ]:
skewness_calificacion = df.select(skewness("Calificacion")).collect()[0][0]
print(skewness_calificacion)

kurtosis_calificacion = df.select(kurtosis("Calificacion")).collect()[0][0]
print(kurtosis_calificacion)

## Graficos

Graficar la distribución de calificaciones

Para esto las pasamos a Pandas y las graficamos utilizando matplotlib

In [ ]:
# Convertir a pandas para graficar
df_pd = df.toPandas()

plt.figure(figsize=(10, 6))
plt.hist(df_pd["Calificacion"], bins=5, edgecolor='k', alpha=0.7)
plt.title("Distribución de Calificaciones")
plt.xlabel("Calificación")
plt.ylabel("Frecuencia")
plt.grid(True)
plt.show()

Graficar la distribución de ingresos

In [ ]:
plt.figure(figsize=(10, 6))
plt.hist(df_pd["Ingreso"], bins=10, edgecolor='k', alpha=0.7)
plt.title("Distribución de Ingresos")
plt.xlabel("Ingreso")
plt.ylabel("Frecuencia")
plt.grid(True)
plt.show()

## Imprimir las métricas

In [ ]:
print(f"Media de Calificación: {mean_calificacion}")
print(f"Mediana de Calificación: {median_calificacion}")
print(f"Moda de Calificación: {mode_calificacion}")
print(f"Desviación Estándar de Calificación: {stddev_calificacion}")
print(f"Asimetría de Calificación: {skewness_calificacion}")
print(f"Curtosis de Calificación: {kurtosis_calificacion}")